In [40]:
# STEP 1 — IMPORT LIBRARIES
# ==============================
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import numpy as np
from tensorflow.keras.preprocessing import image

In [41]:
 #STEP 2 — CONNECT TO GOOGLE DRIVE
from google.colab import drive
drive.mount('/content/drive')

# Path to dataset inside Drive
data_dir = "/content/drive/MyDrive/crop yield dataset/dataset"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
# ==============================
# STEP 3 — LOAD DATASET
# Folder structure:
# dataset/
#   ├── healthy_crop/
#   ├── unhealthy_crop/
#   ├── weeds/
#   ├── dry/
# ==============================
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_data = datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),
    batch_size=8,
    class_mode='categorical',
    subset='training'
)

val_data = datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),
    batch_size=8,
    class_mode='categorical',
    subset='validation'
)

print("Class labels:", train_data.class_indices)


Found 33 images belonging to 4 classes.
Found 7 images belonging to 4 classes.
Class labels: {'dry': 0, 'healthy crop': 1, 'unhealthy crop': 2, 'weeds': 3}


In [43]:
# ==============================
# STEP 4 — BUILD MODEL
# ==============================
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(4, activation='softmax')   # 4 classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     4,735,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,828,868 (18.42 MB)

 Trainable params: 4,828,868 (18.42 MB)

 Non-trainable params: 0 (0.00 B)

In [44]:
# ==============================
# STEP 5 — TRAIN MODEL
# ==============================
history = model.fit(train_data, validation_data=val_data, epochs=10)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 410ms/step - accuracy: 0.1795 - loss: 1.5945 - val_accuracy: 0.2857 - val_loss: 1.3309
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 329ms/step - accuracy: 0.4501 - loss: 1.3615 - val_accuracy: 0.2857 - val_loss: 1.3052
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 503ms/step - accuracy: 0.3272 - loss: 1.3451 - val_accuracy: 0.8571 - val_loss: 1.1147
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 528ms/step - accuracy: 0.5596 - loss: 1.0775 - val_accuracy: 0.5714 - val_loss: 1.0719
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 339ms/step - accuracy: 0.5266 - loss: 1.0420 - val_accuracy: 0.7143 - val_loss: 0.7237
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 338ms/step - accuracy: 0.4988 - loss: 0.8849 - val_accuracy: 0.8571 - val_loss: 0.6374
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 399ms/step - accuracy: 0.6097 - loss: 0.7892 - val_accuracy: 0.5714 - val_loss: 0.8627
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 326ms/step - accuracy: 0.8060 - loss: 0.5176 - val_accuracy: 0.2857 - val_loss:

In [45]:
# ==============================
# STEP 6 — EVALUATE MODEL
# ==============================
loss, acc = model.evaluate(val_data)
print(f"✅ Validation Accuracy: {acc*100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step - accuracy: 0.5714 - loss: 0.7146
✅ Validation Accuracy: 57.14%


In [46]:
# STEP 7 — PREDICT SINGLE IMAGE
# ==============================
# Change the image path to test any image
img_path = "/content/drive/MyDrive/crop yield dataset/dataset/unhealthy crop/u1.jpeg"

img = image.load_img(img_path, target_size=(150,150))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

prediction = model.predict(img_array)
labels = list(train_data.class_indices.keys())

print("Predicted label:", labels[np.argmax(prediction)])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
Predicted label: weeds


In [47]:
!pip install gradio --upgrade --quiet


In [48]:
import gradio as gr
from tensorflow.keras.preprocessing import image
import numpy as np

labels = list(train_data.class_indices.keys())

In [49]:
# Define suggestions for each label
suggestions = {
    "healthy crop": "Maintain current care routine and keep monitoring regularly.",
    "unhealthy crop": "Check for pests or nutrient issues and treat immediately.",
    "weeds": "Remove weeds promptly to prevent crop competition.",
    "dry": "Increase irrigation and improve soil moisture retention."
}



In [50]:
def predict_crop(img):
    if img is None:
        return "Please upload an image first."
    img = img.resize((150, 150))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)
    label = labels[np.argmax(pred)]
    confidence = np.max(pred) * 100
    suggestion = suggestions.get(label, "No suggestion available for this category.")
    return f"Prediction: {label} ({confidence:.2f}%)\n\nSuggestion: {suggestion}"



In [51]:
# ✅ launch with share=True (needed in Colab)
interface = gr.Interface(
    fn=predict_crop,
    inputs=gr.Image(type="pil", label="Upload Crop Image"),
    outputs=gr.Markdown(label="Prediction & Suggestion"),
    title="🌾 AI-Powered Crop Yield Prediction",
    description="Upload a crop image to check its condition and get care suggestions instantly."
)

In [52]:
interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6f68c63fbdfaa87bae.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
